#### PRNG evolution analysis with prediction attacks

In [1]:
#Basic imports for now

# NN-Tools
import numpy as np 
import keras 

# Visualization
from IPython.display import SVG 
from IPython.display import display
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

# Printing
from sympy import *
init_printing(use_latex=True)

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# As we get to each PRNG example, we can import like this
from PRNGs import *
from SeedGenerator import *

t = ticks()
Middle_Square(t, 100)

Middle_Square


['326317487346352596242',
 '480369867631902886315',
 '286919003114065979432',
 '479694032027069402557',
 '623870857135296322027',
 '827293137621700290320',
 '559575362917983186639',
 '847925927895571075878',
 '975651992185285297612',
 '551160034412519277191',
 '336094338546661230421',
 '031177329960614134414',
 '034730077441305156479',
 '790790533247756789009',
 '742715355309747300940',
 '128841782518312578396',
 '224961567357586122675',
 '879817581167122985797',
 '307670432147578864957',
 '178779302925592242299',
 '545606748863811896000',
 '057387036667194076711',
 '774418774435497731902',
 '981783154283133307877',
 '341387401000326781453',
 '617579191418694540921',
 '733685528915202684190',
 '395807139915107577885',
 '077775463971849211325',
 '960364146984643265030',
 '135414936398975881289',
 '999386830197688188583',
 '725828442718937436852',
 '597978438883921272753',
 '700515687729386160532',
 '549748641987367021436',
 '669542383802113719713',
 '074169523106586115098',
 '5785841164